In [ ]:
import os
import sys
from functools import singledispatch
import warnings

import pandas as pd
import numpy as np

sys.path.insert(0, "/home/tthatcher/Desktop/Projects/Plio/plio")

from plio.examples import get_path
from plio.io.io_bae import read_gpf, read_ipf

In [ ]:
# Reads a .atf file and outputs all of the 
# .ipf, .gpf, .sup, .prj, and path to locate the 
# .apf file (should be the same as all others) 
def read_atf(atf_file):
    with open(atf_file) as f:

        files = []
        ipf = []
        sup = []
        files_dict = []
        
        # Grabs every PRJ, GPF, SUP, and IPF image from the ATF file
        for line in f:
            if line[-4:-1] == 'prj' or line[-4:-1] == 'gpf' or line[-4:-1] == 'sup' or line[-4:-1] == 'ipf' or line[-4:-1] == 'atf':
                files.append(line)
        
        files = np.array(files)
        
        # Creates appropriate arrays for certain files in the right format
        for file in files:
            file = file.strip()
            file = file.split(' ')

            # Grabs all the IPF files
            if file[1].endswith('.ipf'):
                ipf.append(file[1])

            # Grabs all the SUP files
            if file[1].endswith('.sup'):
                sup.append(file[1])

            files_dict.append(file)

        # Creates a dict out of file lists for GPF, PRJ, IPF, and ATF
        files_dict = (dict(files_dict))
        
        # Sets the value of IMAGE_IPF to all IPF images
        files_dict['IMAGE_IPF'] = ipf
        
        # Sets the value of IMAGE_SUP to all SUP images
        files_dict['IMAGE_SUP'] = sup
        
        # Sets the value of PATH to the path of the ATF file
        files_dict['PATH'] = os.path.dirname(os.path.abspath(atf_file))
        
        return files_dict

In [ ]:
atf_dict = read_atf(get_path('CTX_Athabasca_Middle_step0.atf'))
print(atf_dict)

gpf_file = os.path.join(atf_dict['PATH'], atf_dict['GP_FILE']);
ipf_list = [os.path.join(atf_dict['PATH'], i) for i in atf_dict['IMAGE_IPF']]

gpf_df = read_gpf(gpf_file).set_index('point_id')
ipf_df = read_ipf(ipf_list).set_index('pt_id')

point_diff = ipf_df.index.difference(gpf_df.index)

if len(point_diff) != 0:
    warnings.warn("The following points found in ipf files missing from gpf file: \n\n{}. \
                  \n\nContinuing, but these points will be missing from the control network".format(list(point_diff)))

new_df = ipf_df.merge(gpf_df, left_index=True, right_index=True)
list(new_df)

In [ ]:
def sample_size(record):
    
    with open(atf_dict['PATH'] + '/' + record['ipf_file'] + '.sup') as f:
        sample_num = 0
        for i, line in enumerate(f):
            if i == 4:
                sample_num = line.split(' ')
                sample_num = sample_num[-1].strip()
                break
        assert int(sample_num) > 0, "Sample number {} from {} is a negative number: Invalid Data".format(sample_num, record['ipf_file'])
            
        sample_num = int(sample_num)/2.0 + record['s.'] + 1
        return sample_num

def line_size(record):
    
    with open(atf_dict['PATH'] + '/' + record['ipf_file'] + '.sup') as f:
        sample_num = 0
        for i, line in enumerate(f):
            if i == 3:
                sample_num = line.split(' ')
                sample_num = sample_num[-1].strip()
                break
        assert int(sample_num) > 0, "Sample number {} from {} is a negative number: Invalid Data".format(sample_num, record['ipf_file'])
            
        sample_num = int(sample_num)/2.0 + record['l.'] + 1
        return sample_num

def known(record):
    if record['known'] == 0:
        return 'Free'
    
    elif record['known'] == 1 or record['known'] == 2 or record['known'] == 3:
        return 'Constrained'
    
# new_df['s.'] = new_df.apply(sample_size, axis=1)
# new_df['l.'] = new_df.apply(line_size, axis=1)
# new_df['known'] = new_df.apply(known, axis=1)
display(new_df)
# cmeas->SetCoordinate(img_samples/2.0+sample+1,img_lines/2.0+line+1);

In [ ]:
import math
import pyproj

# converts +/- 180 system to 0 - 360 system
def to_360(num):
    return num % 360

# ocentric to ographic latitudes
def oc2og(dlat, dMajorRadius, dMinorRadius):
    try:    
        dlat = math.radians(dlat)
        dlat = math.atan(((dMajorRadius / dMinorRadius)**2) * (math.tan(dlat)))
        dlat = math.degrees(dlat)
    except:
        print ("Error in oc2og conversion")
    return dlat

# ographic to ocentric latitudes
def og2oc(dlat, dMajorRadius, dMinorRadius):
    try:
        dlat = math.radians(dlat)
        dlat = math.atan((math.tan(dlat) / ((dMajorRadius / dMinorRadius)**2)))
        dlat = math.degrees(dlat)
    except:
        print ("Error in og2oc conversion")
    return dlat

def get_axis():
    with open(atf_dict['PATH'] + '/' + 'CTX_Athabasca_Middle.prj') as f:
        from collections import defaultdict

        files = defaultdict(list)
        
        for line in f:
            
            ext = line.strip().split(' ')
            files[ext[0]].append(ext[-1])
            
        eRadius = float(files['A_EARTH'][0])
        pRadius = eRadius * (1 - float(files['E_EARTH'][0]))
        
        return eRadius, pRadius
    
# function to convert lat_Y_North to ISIS_lat
def lat_ISIS_coord(record):
    with open(atf_dict['PATH'] + '/' + 'CTX_Athabasca_Middle.prj') as f:
        from collections import defaultdict

        files = defaultdict(list)
        
        for line in f:
            
            ext = line.strip().split(' ')
            files[ext[0]].append(ext[-1])
            
        eRadius = float(files['A_EARTH'][0])
        pRadius = eRadius * (1 - float(files['E_EARTH'][0]))
        ocentric_coord = og2oc(record['lat_Y_North'], eRadius, pRadius)
        coord_360 = to_360(ocentric_coord)
        return coord_360

# function to convert long_X_East to ISIS_lon
def lon_ISIS_coord(record):
    with open(atf_dict['PATH'] + '/' + 'CTX_Athabasca_Middle.prj') as f:
        from collections import defaultdict

        files = defaultdict(list)
        
        for line in f:
            
            ext = line.strip().split(' ')
            files[ext[0]].append(ext[-1])
            
        eRadius = float(files['A_EARTH'][0])
        pRadius = eRadius * (1 - float(files['E_EARTH'][0]))
        ocentric_coord = og2oc(record['long_X_East'], eRadius, pRadius)
        coord_360 = to_360(ocentric_coord)
        return coord_360
            
def body_fixed_lat(record):
    semi_major, semi_minor = get_axis() 
    ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
    lla = pyproj.Proj(proj='latlon', a=semi_major, b=semi_minor)
    lon, lat, height = pyproj.transform(lla, ecef, record['long_X_East'], record['lat_Y_North'], record['ht'])
    return lat

def body_fixed_lon(record):
    semi_major, semi_minor = get_axis() 
    ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
    lla = pyproj.Proj(proj='latlon', a=semi_major, b=semi_minor)
    lon, lat, height = pyproj.transform(lla, ecef, record['long_X_East'], record['lat_Y_North'], record['ht'])
    return lon

def body_fixed_height(record):
    semi_major, semi_minor = get_axis() 
    ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
    lla = pyproj.Proj(proj='latlon', a=semi_major, b=semi_minor)
    lon, lat, height = pyproj.transform(lla, ecef, record['long_X_East'], record['lat_Y_North'], record['ht'])
    return height


new_df['lat_Y_North'] = new_df.apply(lat_ISIS_coord, axis=1)
new_df['long_X_East'] = new_df.apply(lon_ISIS_coord, axis=1)

new_df['lat_Y_North'] = new_df.apply(body_fixed_lat, axis=1)
new_df['long_X_East'] = new_df.apply(body_fixed_lon, axis=1)

new_df['ht'] = new_df.apply(body_fixed_height, axis=1)


display(new_df)

In [ ]:
# Reads a .atf file and outputs all of the 
# .ipf, .gpf, .sup, .prj, and path to locate the 
# .apf file (should be the same as all others) 
def read_atf(atf_file):
    with open(atf_file) as f:

        files = []
        ipf = []
        sup = []
        files_dict = []
        
        # Grabs every PRJ, GPF, SUP, and IPF image from the ATF file
        for line in f:
            if line[-4:-1] == 'prj' or line[-4:-1] == 'gpf' or line[-4:-1] == 'sup' or line[-4:-1] == 'ipf' or line[-4:-1] == 'atf':
                files.append(line)
        
        files = np.array(files)
        
        # Creates appropriate arrays for certain files in the right format
        for file in files:
            file = file.strip()
            file = file.split(' ')

            # Grabs all the IPF files
            if file[1].endswith('.ipf'):
                ipf.append(file[1])

            # Grabs all the SUP files
            if file[1].endswith('.sup'):
                sup.append(file[1])

            files_dict.append(file)

        # Creates a dict out of file lists for GPF, PRJ, IPF, and ATF
        files_dict = (dict(files_dict))
        
        # Sets the value of IMAGE_IPF to all IPF images
        files_dict['IMAGE_IPF'] = ipf
        
        # Sets the value of IMAGE_SUP to all SUP images
        files_dict['IMAGE_SUP'] = sup
        
        # Sets the value of PATH to the path of the ATF file
        files_dict['PATH'] = os.path.dirname(os.path.abspath(atf_file))
        
        return files_dict
    
@singledispatch
def read_ipf(arg):
    return str(arg)
# new_df['known'] = new_df.apply(known, axis=1)

@read_ipf.register(str)
def read_ipf_str(input_data):
    """
    Read a socet ipf file into a pandas data frame

    Parameters
    ----------
    input_data : str
                 path to the an input data file

    Returns
    -------
    df : pd.DataFrame
         containing the ipf data with appropriate column names and indices
    """

    # Check that the number of rows is matching the expected number
    with open(input_data, 'r') as f:
        for i, l in enumerate(f):
            if i == 1:
                cnt = int(l)
            elif i == 2:
                col = l
                break
                
    columns = np.genfromtxt(input_data, skip_header=2, dtype='unicode',
                            max_rows = 1, delimiter = ',')

    # TODO: Add unicode conversion
    d = [line.split() for line in open(input_data, 'r')]
    d = np.hstack(np.array(d[3:]))
    
    d = d.reshape(-1, 12)
    
    df = pd.DataFrame(d, columns=columns)
    file = os.path.split(os.path.splitext(input_data)[0])[1]
    df['ipf_file'] = pd.Series(np.full((len(df['pt_id'])), file), index = df.index)

    assert int(cnt) == len(df), 'Dataframe length {} does not match point length {}.'.format(int(cnt), len(df))
    
    # Soft conversion of numeric types to numerics, allows str in first col for point_id
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

@read_ipf.register(list)
def read_ipf_list(input_data_list):
    """
    Read a socet ipf file into a pandas data frame

    Parameters
    ----------
    input_data_list : list
                      list of paths to the a set of input data files

    Returns
    -------
    df : pd.DataFrame
         containing the ipf data with appropriate column names and indices
    """
    frames = []

    for input_file in input_data_list:
        frames.append(read_ipf(input_file))

    df = pd.concat(frames)

    return df